In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Part 1: Load and Create Dataframe from Wiki Page

### Create a list from the data on wiki page

In [401]:
import pandas as pd    
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'    
toronto_wiki = pd.read_html(page, index_col=0, attrs={"class":"wikitable"})     

AttributeError: 'list' object has no attribute 'head'

### Place list's datas into a dataframe

In [283]:
toronto_wiki_df = pd.DataFrame()
toronto_wiki_df

for data in toronto_wiki:
     toronto_wiki_df= toronto_wiki_df.append(data)
toronto_wiki_df=toronto_wiki_df.reset_index()
toronto_wiki_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Rename "Postcode" to "PostalCode" and "Neighbourhood" to "Neighborhood"

In [402]:
toronto_wiki_df=toronto_wiki_df.rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"})
toronto_wiki_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Turn any non-assigned borough into NaN then drop those data point that have NaN borough

In [285]:
toronto_wiki_df.loc[toronto_wiki_df['Borough'] == 'Not assigned','Borough'] = np.nan
toronto_wiki_df.dropna(inplace=True)

### Simplify the dataframe

In [286]:
toronto_neighborhoods=toronto_wiki_df.groupby('PostalCode').agg({'Borough':'first','Neighborhood': ', '.join})

### Reset index

In [403]:
toronto_neighborhoods=toronto_neighborhoods.reset_index() 
toronto_neighborhoods.head()

,index,PostalCode,Borough,Neighborhood
0,0,M1B,Scarborough,"Rouge, Malvern"
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae


In [289]:
toronto_neighborhoods.shape

(103, 3)

## Part 2: Geographical Coordinates 

In [404]:
geo_data=pd.read_csv("http://cocl.us/Geospatial_data")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [376]:
toronto2=toronto.groupby('PostalCode').agg({'Borough':'first','Neighborhood': ', '.join})
toronto2

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Rouge, Malvern"
M3A,North York,Parkwoods
M3B,North York,Don Mills North
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Downtown Toronto,Queen's Park
M9A,Etobicoke,Islington Avenue


In [377]:
toronto2=toronto2.reset_index()
toronto2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M3A,North York,Parkwoods
2,M3B,North York,Don Mills North
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Downtown Toronto,Queen's Park
7,M9A,Etobicoke,Islington Avenue


In [379]:
toronto3=toronto.groupby('PostalCode').agg({'Borough':'first','Neighborhood': ', '.join})


In [336]:
a=geo_data.loc[geo_data['Postal Code']=="M3A", ['Latitude','Longitude']]
a

,Latitude,Longitude
25,43.753259,-79.329656


In [399]:
toronto3=toronto2
toronto2


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M3A,North York,Parkwoods
2,M3B,North York,Don Mills North
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Downtown Toronto,Queen's Park
7,M9A,Etobicoke,Islington Avenue


In [397]:
c=pd.DataFrame(columns=["Latitude","Longitude"])
toronto3 = pd.concat([toronto3, c], axis=1, sort=False)
toronto3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",NaN,NaN
1,M3A,North York,Parkwoods,NaN,NaN
2,M3B,North York,Don Mills North,NaN,NaN
3,M4A,North York,Victoria Village,NaN,NaN
4,M5A,Downtown Toronto,Harbourfront,NaN,NaN
5,M6A,North York,"Lawrence Heights, Lawrence Manor",NaN,NaN
6,M7A,Downtown Toronto,Queen's Park,NaN,NaN
7,M9A,Etobicoke,Islington Avenue,NaN,NaN


In [398]:
for data in toronto3["PostalCode"]:
    print(data)
    lat=geo_data.loc[geo_data['Postal Code']==data, ['Latitude','Longitude']]
    toronto3.loc[toronto3['PostalCode']==data, ['Latitude','Longitude']]=lat
    print(toronto3)

M1B
  PostalCode           Borough                      Neighborhood Latitude  \
0        M1B       Scarborough                    Rouge, Malvern  43.8067   
1        M3A        North York                         Parkwoods      NaN   
2        M3B        North York                   Don Mills North      NaN   
3        M4A        North York                  Victoria Village      NaN   
4        M5A  Downtown Toronto                      Harbourfront      NaN   
5        M6A        North York  Lawrence Heights, Lawrence Manor      NaN   
6        M7A  Downtown Toronto                      Queen's Park      NaN   
7        M9A         Etobicoke                  Islington Avenue      NaN   

  Longitude  
0  -79.1944  
1       NaN  
2       NaN  
3       NaN  
4       NaN  
5       NaN  
6       NaN  
7       NaN  
M3A
  PostalCode           Borough                      Neighborhood Latitude  \
0        M1B       Scarborough                    Rouge, Malvern  43.8067   
1        M3A      

In [343]:
lat["Latitude"]

93    43.667856
Name: Latitude, dtype: float64

In [369]:
toronto3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M3A,North York,Parkwoods,NaN,NaN
2,M3B,North York,Don Mills North,NaN,NaN
3,M4A,North York,Victoria Village,NaN,NaN
4,M5A,Downtown Toronto,Harbourfront,NaN,NaN
5,M6A,North York,"Lawrence Heights, Lawrence Manor",NaN,NaN
6,M7A,Downtown Toronto,Queen's Park,NaN,NaN
7,M9A,Etobicoke,Islington Avenue,NaN,NaN
